# Data Driven approach using GeoPandas and GTFS
## Pieter Mulder

## What is GeoPandas?

> GeoPandas is an open source project to make working with geospatial data in python easier. GeoPandas extends the datatypes used by pandas to allow spatial operations on geometric types. 

http://geopandas.org/

## What is GTFS?

General Transit Feed Specification

> Defines a common format for public transportation schedules and associated geographic information.

https://developers.google.com/transit/gtfs/

![GTFS Diagram](gtfs-feed-diagram.png "GTFS Diagram")

### Create Panda dataframes of the GTFS data.

In [1]:
import zipfile

import pandas as pd
import geopandas as gpd

zip_obj = zipfile.ZipFile('google_transit.zip')

dtype = {c: str for c in {'agency_id', 'service_id', 'fare_id', 'route_id',
                          'trip_id', 'shape_id', 'stop_id', 'parent_station',
                          'from_stop_id', 'to_stop_id'}}

gtfs = {}
for filename in zip_obj.namelist():
    filelabel = filename.replace('.txt', '')
    gtfs[filelabel] = pd.read_csv(zip_obj.open(filename),
                                  encoding='utf-8-sig', dtype=dtype)
    

### Transform the stops longitude and latitude in python objects we can use.

In [2]:
from shapely.geometry import Point

gtfs['stops'] = gpd.GeoDataFrame(
    gtfs['stops'].assign(
        stop_name=gtfs['stops']['stop_name']\
        .map(lambda x: x[9:].strip() if x.lower().startswith('karlsruhe') else x),
        geometry=gtfs['stops'].apply(
            lambda row: Point(row['stop_lon'], row['stop_lat']), axis=1)
    ).drop(['stop_lat', 'stop_lon'], axis=1)
)
gtfs['stops'].head()

,stop_id,stop_name,zone_id,stop_url,location_type,parent_station,geometry
0,de:07334:1714:1:1,Wörth Alte Bahnmeisterei,NaN,NaN,NaN,Parent1714,POINT (8.26622538039577 49.048742345982)
1,de:07334:1714:1:2,Wörth Alte Bahnmeisterei,NaN,NaN,NaN,Parent1714,POINT (8.266737420107789 49.0484420719247)
2,de:07334:1721:1:1,Maximiliansau Eisenbahnstraße,NaN,NaN,NaN,Parent1721,POINT (8.29789997731824 49.0373071007148)
3,de:07334:1721:2:2,Maximiliansau Eisenbahnstraße,NaN,NaN,NaN,Parent1721,POINT (8.29896897250649 49.0371363175998)
4,de:07334:1723:1:1,Maximiliansau West,NaN,NaN,NaN,Parent1723,POINT (8.292024995359281 49.038861789003)


In [3]:
len('karlsruhe')

9

### Transform the departure and arrival times in values we can use

GTFS knows about times over 24:00. To simplify our problem let's just use integers for the times.

In [4]:
def time_to_int(time):
    return int(time.replace(':', '')[:4])

gtfs['stop_times'] = gtfs['stop_times'].assign(
    arrival_time_int=gtfs['stop_times']['arrival_time'].map(time_to_int),
    departure_time_int=gtfs['stop_times']['departure_time'].map(time_to_int)
)

## Projections

Displaying the (almost) round world on a flat service is hard.

![xkcd](https://imgs.xkcd.com/comics/map_projections.png) 
xkcd.com

Most maps on the internet (and the GTFS) use WGS84.

If we want to measure distances in meters in Europe we can use ETRS89.

### Create the projections

In [5]:
import pyproj

base_proj = pyproj.Proj(init='EPSG:4326')  # WGS84
calc_proj = pyproj.Proj(init='EPSG:3035')  # ETRS89

### Reproject stops to ETRS89

In [6]:
gtfs['stops'].crs = base_proj.srs
gtfs['stops'] = gtfs['stops'].to_crs(calc_proj.srs)

In [7]:
from functools import partial

from shapely.ops import transform

def transform_srs(geometry, *, from_proj=base_proj, to_proj=calc_proj):
    if from_proj == to_proj:
        return geometry
    project = partial(pyproj.transform, from_proj, to_proj)
    geometry = transform(project, geometry)
    return geometry

## Geonotebook

> GeoNotebook is an extension to the Jupyter Notebook that provides interactive visualization and python-based analysis of geospatial data.

http://geonotebook.readthedocs.io/en/latest/


In [8]:
here = Point(8.383590, 49.001763)

M.set_center(here.x, here.y, 13)

### From shapely geometries to points on the map.

In [9]:
def geometry_to_map(geometry, *, from_proj=calc_proj, **kwargs):
    geometry = transform_srs(geometry, to_proj=base_proj,
                             from_proj=from_proj)
    geo_interface = geometry.__geo_interface__
    type_ = geo_interface['type'].lower()
    coords = geo_interface['coordinates']
    if type_ == 'polygon':
        if len(coords) > 1:
            kwargs.setdefault('hole', coords[2:])
        coords = coords[0]
    elif type_ != 'point':
        raise ValueError('Can only show Points and Polygons on map.')
    M.add_annotation(type_, coords, kwargs)

In [10]:
stops = gtfs['stops']

stop_types = {'all', 'parents', 'children'}

def stops_to_map(stops, *, types='parents'):
    if types not in stop_types:
        raise ValueError(
            '"{}" must be one of "{}"'.format(types,
                                              ', '.join(stop_types))
        )
    if types == 'parents':
        stops = stops.dropna(subset=['location_type'])
    elif types == 'children':
        stops = stops.dropna(subset=['parent_station'])
    for _, stop in stops.iterrows():
        geometry_to_map(stop['geometry'], name=stop['stop_name'])
        
stops_to_map(stops.tail(300))

## Let's see where we can get from here.

In [18]:
M.layers.annotation.clear_annotations()

M.set_center(here.x, here.y, 13)

geometry_to_map(here, name='ZKM', from_proj=base_proj)

In [12]:
here_e = transform_srs(here)

In [13]:
walking_speed = 1.39  # m/s = 5 km/h
walking_time = 5 * 60  # 5 minutes
walking_distance = here_e.buffer(walking_speed * walking_time)
geometry_to_map(walking_distance, from_proj=calc_proj, name='wlk_dist')

walkable_stops = stops[stops.intersects(walking_distance)]
stops_to_map(walkable_stops)

In [14]:
# TODO Only for Thursday

In [15]:
stop_times = gtfs['stop_times'].merge(walkable_stops, on='stop_id')
trips = stop_times[
    (stop_times['departure_time_int'] <= 1810) &
    (stop_times['departure_time_int'] >= 1805)
][['trip_id', 'stop_sequence','departure_time']].rename(
    columns={'stop_sequence': 'current_sequence'})

In [20]:
other_stops = gtfs['stop_times'].merge(trips, on='trip_id')
other_stops = other_stops[
    (other_stops['stop_sequence'] > other_stops['current_sequence']) &
    (other_stops['arrival_time_int'] <= 1830)
]

other_stops = stops.merge(
    other_stops[['stop_id', 'arrival_time_int', 'departure_time_int']],
    on='stop_id'
).sort_values(
    ['arrival_time_int']
).drop_duplicates(subset=['stop_id'])
other_stops = other_stops[~other_stops['stop_id'].isin(
    walkable_stops['stop_id']
)]



def get_parents(stops):
    return gtfs['stops'].dropna(
        subset=['location_type']
    ).merge(
        stops.drop_duplicates(subset='parent_station')
        .rename(columns={'parent_station': 'stop_id'}),
        on='stop_id'
    )


parent_stops = get_parents(other_stops[['parent_station', 'arrival_time_int']])
stops_to_map(parent_stops)

for _, stop in parent_stops.iterrows():
    wk_time = min(walking_time, (1830 - stop['arrival_time_int']) * 60)
    if not wk_time:
        continue
    walking_distance = stop['geometry'].buffer(walking_speed * wk_time)
    geometry_to_map(walking_distance, from_proj=calc_proj, name=stop['stop_name'])


In [17]:
gtfs['stops'].groupby('parent_station').size()

# Get all parent stop children
# recursive

parent_station
Parent1        4
Parent10       2
Parent101      2
Parent102      2
Parent103      2
Parent104      2
Parent105      4
Parent106      3
Parent107      3
Parent11       4
Parent1101     4
Parent1103     2
Parent1104     3
Parent1105     3
Parent1106     3
Parent1110     4
Parent1111     4
Parent1114     2
Parent1115     2
Parent1120     2
Parent1122     2
Parent1123     2
Parent1124     2
Parent1131     2
Parent1132     3
Parent1133     2
Parent1140     1
Parent1151     2
Parent1152     2
Parent1153     2
              ..
Parent8902     2
Parent9        2
Parent90       4
Parent90003    1
Parent9001     2
Parent9002     2
Parent9006     2
Parent91       4
Parent9108     2
Parent9109     2
Parent9110     2
Parent9112     2
Parent9116     2
Parent9119     2
Parent9209     2
Parent9210     2
Parent9211     2
Parent9212     2
Parent9312     2
Parent9402     2
Parent9405     2
Parent9410     2
Parent9412     2
Parent98       4
Parent99       4
Parent991      1
Parent99103    2